Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recommending movies: retrieval using a sequential model

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/examples/sequential_retrieval"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/sequential_retrieval.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/sequential_retrieval.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/sequential_retrieval.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this tutorial, we are going to build a sequential retrieval model. Sequential recommendation is a popular model that looks at a sequence of  items that users have interacted with previously and then predicts the next item. Here the order of the items within each sequence matters, so we are going to use a recurrent neural network to model the sequential relationship. For more details, please refer to this [GRU4Rec paper](https://arxiv.org/abs/1511.06939).



## Imports

First let's get our dependencies and imports out of the way.

In [2]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

## Preparing the dataset

Next, we need to prepare our dataset. We are going to leverage the [data generation utility](https://github.com/tensorflow/examples/blob/master/lite/examples/recommendation/ml/data/example_generation_movielens.py) in this [TensorFlow Lite On-device Recommendation reference app](https://www.tensorflow.org/lite/examples/recommendation/overview).

MovieLens 1M data contains ratings.dat (*columns: UserID, MovieID, Rating, Timestamp*), and movies.dat (*columns: MovieID, Title, Genres*). The example generation script download the 1M dataset, takes both files, only keep ratings higher than 2, form user movie interaction timelines, sample activities as labels and 10 previous user activities as the context for prediction.

In [4]:
!wget -nc https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/recommendation/ml/data/example_generation_movielens.py
!python -m example_generation_movielens  --data_dir=data/raw  --output_dir=data/examples  --min_timeline_length=3  --max_context_length=10  --max_context_movie_genre_length=10  --min_rating=2  --train_data_fraction=0.9  --build_vocabs=False

--2022-07-16 11:17:48--  https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/recommendation/ml/data/example_generation_movielens.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 18041 (18K) [text/plain]
Saving to: ‘example_generation_movielens.py’

example_generation_ 100%[===================>]  17.62K  --.-KB/s    in 0.001s  

2022-07-16 11:17:49 (25.2 MB/s) - ‘example_generation_movielens.py’ saved [18041/18041]



I0716 11:17:51.181339 140695481390912 example_generation_movielens.py:460] Downloading and extracting data.


   8192/5917549 [..............................] - ETA: 0s

 155648/5917549 [..............................] - ETA: 2s

...

85220/93799 [==========================>...] - ETA: 0s

88212/93799 [===========================>..] - ETA: 0s

91158/93799 [============================>.] - ETA: 0s

93799/93799 [==============================] - 2s 17us/step


I0716 11:19:39.656523 140695481390912 example_generation_movielens.py:473] Generated dataset: {'train_size': 844195, 'test_size': 93799, 'train_file': 'data/examples/train_movielens_1m.tfrecord', 'test_file': 'data/examples/test_movielens_1m.tfrecord'}


Here is a sample of the generated dataset.

```
0 : {
  features: {
    feature: {
      key  : "context_movie_id"
      value: { int64_list: { value: [ 1124, 2240, 3251, ..., 1268 ] } }
    }
    feature: {
      key  : "context_movie_rating"
      value: { float_list: {value: [ 3.0, 3.0, 4.0, ..., 3.0 ] } }
    }
    feature: {
      key  : "context_movie_year"
      value: { int64_list: { value: [ 1981, 1980, 1985, ..., 1990 ] } }
    }
    feature: {
      key  : "context_movie_genre"
      value: { bytes_list: { value: [ "Drama", "Drama", "Mystery", ..., "UNK" ] } }
    }
    feature: {
      key  : "label_movie_id"
      value: { int64_list: { value: [ 3252 ] }  }
    }
  }
}
```
You can see that it includes a sequence of context movie IDs, and a label movie ID (next movie), plus context features such as movie year, rating and genre. 

In our case we will only be using the sequence of context movie IDs and the label movie ID. You can refer to the [Leveraging context features tutorial](https://www.tensorflow.org/recommenders/examples/context_features) to learn more about adding additional context features.

In [5]:
train_filename = "./data/examples/train_movielens_1m.tfrecord"
train = tf.data.TFRecordDataset(train_filename)

test_filename = "./data/examples/test_movielens_1m.tfrecord"
test = tf.data.TFRecordDataset(test_filename)

feature_description = {
    'context_movie_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_movie_rating': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_movie_year': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(1980, 10)),
    'context_movie_genre': tf.io.FixedLenFeature([10], tf.string, default_value=np.repeat("Drama", 10)),
    'label_movie_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

def _parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, feature_description)

train_ds = train.map(_parse_function).map(lambda x: {
    "context_movie_id": tf.strings.as_string(x["context_movie_id"]),
    "label_movie_id": tf.strings.as_string(x["label_movie_id"])
})

test_ds = test.map(_parse_function).map(lambda x: {
    "context_movie_id": tf.strings.as_string(x["context_movie_id"]),
    "label_movie_id": tf.strings.as_string(x["label_movie_id"])
})

for x in train_ds.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'context_movie_id': array([b'908', b'2294', b'3812', b'0', b'0', b'0', b'0', b'0', b'0', b'0'],
      dtype=object),
 'label_movie_id': array([b'3835'], dtype=object)}


Now our train/test datasets include only a sequence of historical movie IDs and a label of next movie ID. Note that we use `[10]` as the shape of the features during tf.Example parsing because we specify 10 as the length of context features in the example generateion step.

We need one more thing before we can start building the model - the vocabulary for our movie IDs.

In [6]:
movies = tfds.load("movielens/1m-movies", split='train')
movies = movies.map(lambda x: x["movie_id"])
movie_ids = movies.batch(1_000)
unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))

## Implementing a sequential model

In our [basic retrieval tutorial](https://www.tensorflow.org/recommenders/examples/basic_retrieval), we use one query tower for the user, and the candidate tow for the candidate movie. However, the two-tower architecture is generalizble and not limited to <user,item> pair. You can also use it to do item-to-item recommendation as we note in the [basic retrieval tutorial](https://www.tensorflow.org/recommenders/examples/basic_retrieval#item-to-item_recommendation).

Here we are still going to use the two-tower architecture. Specificially, we use the query tower with a [Gated Recurrent Unit (GRU) layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU) to encode the sequence of historical movies, and keep the same candidate tower for the candidate movie. 

In [7]:
embedding_dimension = 32

query_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_movie_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension), 
    tf.keras.layers.GRU(embedding_dimension),
])

candidate_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension)
])

The metrics, task and full model are defined similar to the basic retrieval model. 

In [8]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(candidate_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

class Model(tfrs.Model):

    def __init__(self, query_model, candidate_model):
        super().__init__()
        self._query_model = query_model
        self._candidate_model = candidate_model

        self._task = task

    def compute_loss(self, features, training=False):
        watch_history = features["context_movie_id"]
        watch_next_label = features["label_movie_id"]

        query_embedding = self._query_model(watch_history)       
        candidate_embedding = self._candidate_model(watch_next_label)
        
        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)

## Fitting and evaluating

We can now compile, train and evaluate our sequential retrieval model.

In [9]:
model = Model(query_model, candidate_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [10]:
cached_train = train_ds.shuffle(10_000).batch(12800).cache()
cached_test = test_ds.batch(2560).cache()

In [11]:
model.fit(cached_train, epochs=3)

Epoch 1/3


      1/Unknown - 4s 4s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 121051.9453 - regularization_loss: 0.0000e+00 - total_loss: 121051.9453

      2/Unknown - 4s 223ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 121045.2461 - regularization_loss: 0.0000e+00 - total_loss: 121045.2461

      3/Unknown - 5s 221ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 120981.3880 - regularization_loss: 0.0000e+00 - total_loss: 120981.3880

      4/Unknown - 5s 224ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 121814.9883 - regularization_loss: 0.0000e+00 - total_loss: 121814.9883

...

59/67 [=========================>....] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 99845.6822 - regularization_loss: 0.0000e+00 - total_loss: 99845.6822

61/67 [==========================>...] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 99829.4278 - regularization_loss: 0.0000e+00 - total_loss: 99829.4278

63/67 [===========================>..] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 99809.1037 - regularization_loss: 0.0000e+00 - total_loss: 99809.1037

65/67 [============================>.] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 99792.4941 - regularization_loss: 0.0000e+00 - total_loss: 99792.4941

67/67 [==============================] - 3s 38ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 99618.9623 - regularization_loss: 0.0000e+00 - total_loss: 99618.9623


In [12]:
model.evaluate(cached_test, return_dict=True)

      1/Unknown - 1s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0121 - factorized_top_k/top_5_categorical_accuracy: 0.0664 - factorized_top_k/top_10_categorical_accuracy: 0.1223 - factorized_top_k/top_50_categorical_accuracy: 0.3715 - factorized_top_k/top_100_categorical_accuracy: 0.5004 - loss: 15861.9629 - regularization_loss: 0.0000e+00 - total_loss: 15861.9629

      2/Unknown - 2s 225ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0143 - factorized_top_k/top_5_categorical_accuracy: 0.0711 - factorized_top_k/top_10_categorical_accuracy: 0.1262 - factorized_top_k/top_50_categorical_accuracy: 0.3676 - factorized_top_k/top_100_categorical_accuracy: 0.4990 - loss: 15884.0010 - regularization_loss: 0.0000e+00 - total_loss: 15884.0010

      3/Unknown - 2s 216ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0143 - factorized_top_k/top_5_categorical_accuracy: 0.0728 - factorized_top_k/top_10_categorical_accuracy: 0.1285 - factorized_top_k/top_50_categorical_accuracy: 0.3701 - factorized_top_k/top_100_categorical_accuracy: 0.5043 - loss: 15874.4131 - regularization_loss: 0.0000e+00 - total_loss: 15874.4131

      4/Unknown - 2s 218ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0142 - factorized_top_k/top_5_categorical_accuracy: 0.0745 - factorized_top_k/top_10_categorical_accuracy: 0.1321 - factorized_top_k/top_50_categorical_accuracy: 0.3710 - factorized_top_k/top_100_categorical_accuracy: 0.5036 - loss: 15857.2605 - regularization_loss: 0.0000e+00 - total_loss: 15857.2605

      5/Unknown - 2s 216ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0145 - factorized_top_k/top_5_categorical_accuracy: 0.0752 - factorized_top_k/top_10_categorical_accuracy: 0.1341 - factorized_top_k/top_50_categorical_accuracy: 0.3741 - factorized_top_k/top_100_categorical_accuracy: 0.5059 - loss: 15830.3332 - regularization_loss: 0.0000e+00 - total_loss: 15830.3332

...

     35/Unknown - 9s 214ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0143 - factorized_top_k/top_5_categorical_accuracy: 0.0772 - factorized_top_k/top_10_categorical_accuracy: 0.1353 - factorized_top_k/top_50_categorical_accuracy: 0.3718 - factorized_top_k/top_100_categorical_accuracy: 0.5048 - loss: 15831.1329 - regularization_loss: 0.0000e+00 - total_loss: 15831.1329

     36/Unknown - 9s 214ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0143 - factorized_top_k/top_5_categorical_accuracy: 0.0773 - factorized_top_k/top_10_categorical_accuracy: 0.1353 - factorized_top_k/top_50_categorical_accuracy: 0.3718 - factorized_top_k/top_100_categorical_accuracy: 0.5051 - loss: 15831.3126 - regularization_loss: 0.0000e+00 - total_loss: 15831.3126

     37/Unknown - 9s 212ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0143 - factorized_top_k/top_5_categorical_accuracy: 0.0775 - factorized_top_k/top_10_categorical_accuracy: 0.1355 - factorized_top_k/top_50_categorical_accuracy: 0.3718 - factorized_top_k/top_100_categorical_accuracy: 0.5053 - loss: 15655.6869 - regularization_loss: 0.0000e+00 - total_loss: 15655.6869

37/37 [==============================] - 9s 212ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0143 - factorized_top_k/top_5_categorical_accuracy: 0.0775 - factorized_top_k/top_10_categorical_accuracy: 0.1355 - factorized_top_k/top_50_categorical_accuracy: 0.3718 - factorized_top_k/top_100_categorical_accuracy: 0.5053 - loss: 15489.3047 - regularization_loss: 0.0000e+00 - total_loss: 15489.3047


{'factorized_top_k/top_1_categorical_accuracy': 0.014339172281324863,
 'factorized_top_k/top_5_categorical_accuracy': 0.07747417688369751,
 'factorized_top_k/top_10_categorical_accuracy': 0.13549184799194336,
 'factorized_top_k/top_50_categorical_accuracy': 0.37182700634002686,
 'factorized_top_k/top_100_categorical_accuracy': 0.505261242389679,
 'loss': 9333.162109375,
 'regularization_loss': 0,
 'total_loss': 9333.162109375}

This concludes the sequential retrieval tutorial.